In [1]:
# GPU: 32*40 in 27.9s = 46/s
# CPU: 32*8 in 70s = 4/s

In [2]:
import os
# TOGGLE THIS (hard to kill CNTK):
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# ...
os.environ['KERAS_BACKEND'] = "cntk"
import sys
import numpy as np
import keras as K
import cntk as C
from keras.applications.resnet50 import ResNet50
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Keras: ", K.__version__)
print("CNTK: ", C.__version__)
print("Keras using {}".format(K.backend.backend()))
print("Keras channel ordering is {}".format(K.backend.image_data_format()))

Using CNTK backend


OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.3
Keras:  2.0.9
CNTK:  2.2
Keras using cntk
Keras channel ordering is channels_last


/anaconda/envs/py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:18: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


In [3]:
!cat /proc/cpuinfo | grep processor | wc -l

6


In [4]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla K80


In [5]:
# Set channels first (default)
K.backend.set_image_data_format('channels_first')
BATCH_SIZE = 32
RESNET_FEATURES = 2048
BATCHES_GPU = 40
BATCHES_CPU = 8

In [6]:
def give_fake_data(batches):
    """ Create an array of fake data to run inference on"""
    np.random.seed(0)
    dta = np.random.rand(BATCH_SIZE*batches, 224, 224, 3).astype(np.float32)
    return dta, np.swapaxes(dta, 1, 3)

In [7]:
def yield_mb(X, batchsize):
    """ Function yield (complete) mini_batches of data"""
    for i in range(len(X)//batchsize):
        yield i, X[i*batchsize:(i+1)*batchsize]

In [8]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [9]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        out[idx*batchsize:(idx+1)*batchsize] = classifier.predict_on_batch(dta).squeeze()
    return out

## 1. GPU

In [10]:
# Download Resnet weights
model = ResNet50(include_top=False, input_shape=(3,224,224))
#model.summary()

In [11]:
fake_input_data_cf = np.ascontiguousarray(fake_input_data_cf)
cold_start = predict_fn(model, fake_input_data_cf, BATCH_SIZE)

In [12]:
%%time
# GPU: 27.9s
features = predict_fn(model, fake_input_data_cf, BATCH_SIZE)

CPU times: user 24.9 s, sys: 1.17 s, total: 26.1 s
Wall time: 27.9 s


## 2. CPU

In [10]:
# Need to restart notebook

In [11]:
# Kill all GPUs ...
# Done at top of script on re-run (before loading CNTK)

In [12]:
model = ResNet50(include_top=False, input_shape=(3,224,224))

In [13]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCHES_CPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(256, 224, 224, 3) (256, 3, 224, 224)


In [14]:
fake_input_data_cf = np.ascontiguousarray(fake_input_data_cf)
cold_start = predict_fn(model, fake_input_data_cf, BATCH_SIZE)

In [15]:
%%time
# CPU: 70s
features = predict_fn(model, fake_input_data_cf, BATCH_SIZE)

CPU times: user 6min 11s, sys: 7.9 s, total: 6min 19s
Wall time: 1min 10s
